In [1]:
!pip install paramiko
!pip install sshtunnel
!pip install openpyxl
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import paramiko
from sshtunnel import SSHTunnelForwarder
import mysql.connector
import pandas as pd
import warnings

In [6]:
from string import Template

template_invoice = Template("""
SELECT i.id, i.invoice_date, i.invoice_number, i.is_initial, i.lease_id, i.cashflow_id, i.status_id, t.dMonth,t.deleted_at, t.re_sync_status
FROM invoice i LEFT JOIN totalleasecf t ON i.cashflow_id = t.idTotalLeaseCF
WHERE i.lease_id = $id AND i.is_initial = 1 AND i.status_id = 1;
""")

template_tlcf = Template("""
SELECT *
FROM totalleasecf
WHERE idLease = $id and is_initial = 1 and dMonth = $period and re_sync_status = 'Active' and deleted_at is null
""")

template_update = Template("""
UPDATE invoice SET cashflow_id = $new_cashflow_id WHERE id = $invoice_id
""")

In [21]:
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

class IdTotalLeaseCfReassigner:

    def __init__(self, id_lease_to_check: int) -> None:
        self.valores = {'id': id_lease_to_check,
                        'period': '',
                        'dMonth': None,
                        'new_cashflow_id': None,
                        'old_cashflow_id': None,
                        'invoice_id': None
                        }

    def ejecutar_update(self, query):
        try:
            conexion = mysql.connector.connect(
                host="localhost",
                user="idiaz",
                password="FI1cYZAN2ICKr7Ads1RaNGgL8lnX",
                database="quantum"
            )

            cursor = conexion.cursor()
            cursor.execute(query)
            conexion.commit()
            print("Query ejecutada exitosamente")

        except mysql.connector.Error as error:
            print("Error al ejecutar el query:", error)
        finally:
            if conexion.is_connected():
                cursor.close()
                conexion.close()


    def ejecutar_select_to_df(self, query):
        try:
            conexion = mysql.connector.connect(
                host="localhost",
                user="idiaz",
                password="FI1cYZAN2ICKr7Ads1RaNGgL8lnX",
                database="quantum"
            )

            df_result = pd.read_sql(query, conexion)
            print("Query ejecutada exitosamente")
            return df_result
        except mysql.connector.Error as error:
            print("Error al ejecutar el query:", error)
            return None
        finally:
            if conexion.is_connected():
                conexion.close()

    def main(self):
        query =  template_invoice.substitute(self.valores)

        df = self.ejecutar_select_to_df(query)
        self.valores['invoice_id'] = df['id'][0]

        if df['re_sync_status'][0] =='Canceled':
            display(df)
            self.valores['old_cashflow_id'] = df['cashflow_id'][0]
            self.valores['period'] = df['dMonth'][0]
            df_tlcf = self.ejecutar_select_to_df(template_tlcf.substitute(self.valores))
            display(df_tlcf)


            self.valores['new_cashflow_id'] = df_tlcf['idTotalLeaseCF'][0]

            update_q = template_update.substitute(self.valores)
            self.ejecutar_update(update_q)

            display(self.ejecutar_select_to_df(query))
            print(f"""
            Factura de iniciales con invoice_id = {self.valores['invoice_id']} para el id_lease {self.valores['id']}
            Ha sido asignado al Cashflow_id = {self.valores['new_cashflow_id']}
            Anteriormente se tenia asignado al Cashflow_id = {self.valores['old_cashflow_id']}
            """)
        else:
            print(f"No hay facturas de iniciales para el id lease {self.valores['id']}")


In [13]:
def main():
    # Tu código principal va aquí
    ids_to_check = [520]

    for id in ids_to_check:
        driver = IdTotalLeaseCfReassigner(id)
        driver.main()

In [23]:
from IPython.core.display import display, HTML

# Configura para mostrar el contenido completo sin truncarlo
display(HTML("<style>.container { width:100% !important; }</style>"))

ssh_host = 'quantum-v2-devprod-brdg-lb-09381999897f2232.elb.us-east-1.amazonaws.com'
ssh_port = 22
ssh_user = 'ubuntu'
ssh_password = 'heJQAshymxMMhnKW6vth3LlEWbsc'

# Establecer la configuración para el túnel SSH
# v v v v v v v v Cambiar a Prod v v v v v v v v
remote_host = 'quantum-v2-dev-rds.cluster-cegv0wkgdkwr.us-east-1.rds.amazonaws.com'
remote_port = 3306
local_port = 3306

# Crear una instancia de SSHTunnelForwarder para gestionar el túnel SSH
with SSHTunnelForwarder((ssh_host, ssh_port),
                        ssh_username = ssh_user, 
                        ssh_password = ssh_password, 
                        remote_bind_address = (remote_host, remote_port),
                        local_bind_address=('localhost', local_port)) as tunnel:
    main()

Query ejecutada exitosamente
No hay facturas de iniciales para el id lease 520
